In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from datetime import datetime


df_original = pd.read_csv('events.csv', low_memory=False)
display(df_original.head())
df_original.timestamp = df_original.timestamp.astype(datetime)

In [ ]:
df_leads = df_original.loc[df_original['event']=='lead',['model', 'person', 'timestamp']]
display(df_leads.head())
df_bought = df_original.loc[df_original['event']=='conversion', ['model', 'person', 'timestamp']]
display(df_bought.head())

In [ ]:
plt.figure(figsize=(10,8))
top_10_leads = df_leads['model'].value_counts().head(10)
top_10_leads.plot.bar()
plt.xlabel('Modelos de telefonos', fontsize=15)
plt.ylabel('Cantidad de leads asociados', fontsize=15)
plt.title('Cantidad de veces puesto en lead por modelo de telefono', fontsize=20)

In [ ]:
df_bought_from_lead = pd.merge(df_leads, df_bought, how='inner', on=['person', 'model'])
display(df_bought_from_lead.head())
df_bought_from_lead.columns = list(['model', 'person', 'lead_time', 'bought_time'])
display(df_bought_from_lead.describe())
display(df_bought_from_lead.head(10))

In [ ]:
df_bought_from_lead['bought_time'] = df_bought_from_lead['bought_time'].astype(datetime)
df_bought_from_lead['lead_time'] = df_bought_from_lead['lead_time'].astype(datetime)


def time_diff(row):
    bought_date = datetime(*map(int, row['bought_time'].split(' ')[0].split('-')))
    lead_date = datetime(*map(int, row['lead_time'].split(' ')[0].split('-')))
    
    delta = bought_date - lead_date
    return delta.days


df_bought_from_lead['days_elapsed'] = df_bought_from_lead.agg(time_diff, axis=1)

In [ ]:
df_bought_from_lead.head()

Como se puede observar, hay personas que hicieron checkouts y luego leads. Para el siguiente análisis vamos a descartar estas situaciones.

In [ ]:
df_bought_from_lead = df_bought_from_lead.loc[df_bought_from_lead['days_elapsed'] > 0, :]

In [ ]:
df_bought_from_lead

In [ ]:
df_bought_from_lead.days_elapsed.describe()

Se puede observar en estas estadísticas que la mayoría de los que usan la feature de la notificación de stock no tardan mucho en comprar el teléfono que querían. 

Analizando mas de cerca, el usuario que más tardó en comprar el telefóno que quería (7d4cf95d) vemos que pidió que lo notifiquen 2 veces antes de comprar 2 equipos del mismo modelo. Podríamos asumir que no es que el usuario se olvidó o _colgó_, sino que se tardó en volver a tener stock de ese modelo sabiendo que _Motorola Moto G4 Plus_ no es uno de los modelos mas populares en la página.

In [ ]:
plt.figure(figsize=(10,8))
top_10_leads = df_bought_from_lead['model'].value_counts().head(10)
top_10_leads.plot.bar()
plt.xlabel('Modelos de telefonos', fontsize=15)
plt.ylabel('Cantidad de equipos comprados', fontsize=15)
plt.title('Cantidad telefonos comprados que fueron puestos en lead', fontsize=20)

In [ ]:
persons_buying_leads = df_bought_from_lead.groupby(['person', 'model']).agg({'days_elapsed':'count'})
persons_buying_leads = persons_buying_leads.reset_index()
persons_buying_leads.columns = ['person', 'model', 'phones_bought']
persons_buying_leads.sort_values(by='person', inplace=True)
persons_buying_leads

Se puede ver algo interesante, que todas las personas que compran varios equipos usando el sistema de lead, siempre compran el mismo modelo de equipo.